Adjust all necessary variables below for the notebook to properly execute. This includes that the input data are available (see other notebook that must be executed before). There are some additional choice in the third python code cell to select a pmode for the type of baseline fit to be displayed. The user has to play around to find a good choice. Good luck.

In [ ]:
%matplotlib nbagg
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
import os
import sys
from STO2_v35 import *


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
plt.rcParams['axes.formatter.useoffset'] = False
plt.rc("font", size=12)


def resampleElements(aa1, aa2, nn):
    rdd1 = np.nanmean(aa1[0:aa1.size//nn*nn].reshape((aa1.size//nn,nn)), axis=1)
    rdd2 = np.nanmean(aa2[0:aa2.size//nn*nn].reshape((aa2.size//nn,nn)), axis=1)
    return rdd1, rdd2


sto2_plot_path = './Data/plots/'
ofileroot = './Data/processed/EtaCar_'

pos0 = SkyCoord(287.4081*u.deg, -00.5774*u.deg, frame='galactic')   # from Matsuo et al. 2009
l0 = pos0.galactic.l.degree
b0 = pos0.galactic.b.degree

ifile = './Data/processed/EtaCar_map1_0.7vp36_CII_2_3801_3993_v2_20200519.npz'


# detect the line reducced
if 'NII_1' in ifile: lin = 0
if 'NII_2' in ifile: lin = 1
if 'CII_2' in ifile: lin = 2
    
    

dat = np.load(ifile)
sdat = dat['sdata']

skeys = sdat.dtype.names

# for sk in skeys:
#     print(sk, sdat[sk].shape)

vv = np.squeeze(sdat['vv'])
spec = np.squeeze(sdat['spec'])
asp  = np.squeeze(sdat['asp'])
bsp  = np.squeeze(sdat['bsp'])
csp  = np.squeeze(sdat['csp'])
dsp  = np.squeeze(sdat['dsp'])
ref1 = np.squeeze(sdat['ref1'])
ref2 = np.squeeze(sdat['ref2'])
bcor1 = np.squeeze(sdat['bcor1'])
bcor2 = np.squeeze(sdat['bcor2'])
Tsys1 = np.squeeze(sdat['Tsys1'])
Tsys2 = np.squeeze(sdat['Tsys2'])
scan = np.squeeze(sdat['scan'])
obsid = np.squeeze(sdat['sobsid'])
gl = np.squeeze(sdat['gl'])
gb = np.squeeze(sdat['gb'])
mask = np.squeeze(sdat['mask'])   # pixel mask of on-scan
cmask = np.squeeze(sdat['cmask']) # pixel mask of all scans contributing to spectrum
amask = np.squeeze(sdat['amask']) # scan mask of all scans contributing to spectrum


In [ ]:
# read Youngmin's data

import numpy as np

yipath = '/Volumes/rdat/Projects/STO-2/Data/Youngmin/data_20181206/'
yifile = 'Youngmin_cal_3rd_all_raw_compressed.npz'

with np.load(yipath+yifile, allow_pickle=True) as idat:
    print(idat.files)
    dds = idat['dds']
    hds = idat['hds']

# number of observations
nys = hds.shape[0]

print('dds shape: ', dds.shape)
print('hds shape: ', hds.shape)
print('hds type: ', type(hds))
#print(hds)


# create spectra and velocity variables
ysp = np.squeeze(np.array(dds['DATA'], dtype=np.float))
print(ysp.shape)

# number of pixels in spectrum
n_pixl = ysp.shape[2]

yvv = np.zeros(ysp.shape, dtype=np.float)
print(yvv.shape)
for i in range(nys):
    for j in range(3):
        crval1 = np.squeeze(np.array(hds[i]['CRVAL1'], dtype=np.float))
        crpix1 = np.array(dds[i]['CRPIX1'][j], dtype=np.float)
        cdelt1 = np.array(dds[i]['CDELT1'][j], dtype=np.float)
        yvv[i,j,:] = np.squeeze((crval1 + (1 + np.arange(n_pixl) - crpix1) * cdelt1)) / 1000.# * u.km / u.s)

# now, we have to perform a cross reference for the spectra. 

yscans = np.zeros(nys, dtype=np.int)
yobsids = np.zeros(nys, dtype=np.int)
for i in range(nys): 
    yscans[i] = np.int(hds[i]['SCAN'])
    yobsids[i] = np.int(hds[i]['OBSID'])
print(yscans)
print(yobsids)

# array containing the cross references to spectra for comparison
ncr = scan.size
cr = np.zeros(ncr, dtype=int)

for i in range(ncr):
    try:
        cr[i] = np.argwhere((scan[i]==yscans)&(obsid[i]==yobsids))[0]
    except:
        pass
#print(cr.tolist())

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
from astropy.utils.console import ProgressBar
from ALSFitter import *

# plot all spectra of single OTF scan
# one scan per page

plotys = False



bplot = 'Tsys'   #'Tsys'  # could be 'psd', 'ref', 'bcor, or 'Tsys'
vrange  = np.array([-95, 45.]) # velocity range for display
lrange1 = np.array([-30., -5.])   # velocity range covering a line (used for baseline fit)
pxrange = np.array([-120,60])   # plot/display x-range
if lin==2:
    pxrange = np.array([-85,45])    # plot/display x-range
    pyrange = np.array([-10,30.])   # plot/display y-range
    ptx0 = pxrange[0]+10
    pty0 = 25
else:
    pxrange = np.array([-85,35])    # plot/display x-range
    pyrange = np.array([-10,10.])   # plot/display y-range
    ptx0 = pxrange[0]+10
    pty0 = 7
order1 = 13
tsz = 8          # plot label text size
add = '_'+bplot
# p_mode: available: db, base_a, base_b, base_c, base_d (default, best so far!)
pmode = 'base_d' # available: base_a, base_b, base_c, base_d (default, best so far!)
pmode = 'bd' 
pmode = 'acd'
pmode = 'appl_d'
add += '_'+pmode

# select the OTF scans to be plotted; scans must be an array
scans = np.unique(scan)
#scans = scans[0:5]
#scans = [3819]
#scans = [3981]

# more ancillary on-screen output
debug = False
debug = True

# overplot Youngmin's spectra for comparison
plotys = True

if (lin==0)|(lin==1): plotys = False  # the data are not in the file loaded above!!!!
if plotys: add += '_wYS'

outfile = ifile+add+'.pdf'
print('Creating file: %s'%(outfile))

pdf = PdfPages(outfile)

for iscan in ProgressBar(scans, ipython_widget=True):
    
    isel = np.where(scan==iscan)
    iasel = np.argwhere(scan==iscan)

    n_sc = scan[isel].size
    n_cols = 3  # was 5??
    n_rows = np.int(np.ceil(n_sc/3))
    print(iscan)
    if debug: print(iscan, n_sc, n_sc//3+2, n_rows)
        
    tidx = np.hstack([np.arange(0,n_sc,3),np.arange(1,n_sc+1,3),np.arange(2,n_sc+2,3)])


    scvv = np.squeeze(vv[iasel,:])
    scsp = np.squeeze(spec[isel,:])
    # retrieve the full-resolution baseline fits
    scba = np.squeeze(asp[isel,:])   # baseline correction low order
    scbb = np.squeeze(bsp[isel,:])   # baseline correction medium order
    scbc = np.squeeze(csp[isel,:])   # baseline correction aggressive order
    scbd = np.squeeze(dsp[isel,:])   # baseline correction low order without line range
    #
    scr1 = np.squeeze(ref1[isel,:])
    scr2 = np.squeeze(ref2[isel,:])
    sct1 = np.squeeze(Tsys1[isel,:])
    sct2 = np.squeeze(Tsys2[isel,:])
    scgl = np.squeeze(gl[isel])
    scgb = np.squeeze(gb[isel])
    scid = np.squeeze(obsid[isel])
    scmk = np.squeeze(mask[isel,:])
    sccm = np.squeeze(cmask[isel,:])
    scam = np.squeeze(amask[isel,:])
    if debug: print('scsp, iscan, n_sc', scsp.shape, iscan, n_sc)
    
    fig = plt.figure(figsize=(10,20))
    ctitle = 'Eta Car2 Scan %i'%(iscan)
    plt.figtext(0.5, 0.97,ctitle, ha='center',va='bottom')
    #plt.title(ctitle)
    for i in range(n_sc):
        # we have to create a second reordered index counter for 3 -column plots
        spx = plt.subplot(n_sc//3+3,3,tidx[i]+1)
        
        # set all masked pixels to nan except do not use masks of scans that are bad
        # this does not work with the baseline fitting anymore -> find work-around!!!
        # scsp[i,np.where(np.sum(np.squeeze(sccm[i,:][:,np.where(scam[i,:]==0)]),axis=1)>0)] = np.NaN  # remove the spikes!
        
        # print(iscan, i, (scsp[i,vsel][0]).min(), (scsp[i,vsel][0]).max(), (scsp[i,vsel][0]).mean())
        
        vsel = np.where((scvv[i,:]>=vrange[0])&(scvv[i,:]<=vrange[1]))

        if pmode=='db':
            add += '_blad'
            plt.plot(scvv[i,vsel][0], scsp[i,vsel][0], drawstyle='steps-post', color='blue', alpha=0.99, label='spectrum')
            plt.plot(scvv[i,vsel][0], scba[i,vsel][0], drawstyle='steps-post', color='orange', alpha=0.9, label='bline a')
            #plt.plot(scvv[i,vsel][0], scbb[i,vsel][0], drawstyle='steps-post', color='red', alpha=0.7, label='bline b')
            #plt.plot(scvv[i,vsel][0], scbc[i,vsel][0], drawstyle='steps-post', color='green', alpha=0.5, label='bline c')
            plt.plot(scvv[i,vsel][0], scbd[i,vsel][0], drawstyle='steps-post', color='olive', alpha=0.7, label='bline d', linewidth=2.0)
        if pmode=='acd':
            add += '_blacd'
            plt.plot(scvv[i,vsel][0], scsp[i,vsel][0], drawstyle='steps-post', color='blue', alpha=0.99, label='spectrum')
            plt.plot(scvv[i,vsel][0], scba[i,vsel][0], drawstyle='steps-post', color='orange', alpha=0.9, label='bline a')
            #plt.plot(scvv[i,vsel][0], scbb[i,vsel][0], drawstyle='steps-post', color='red', alpha=0.7, label='bline b')
            plt.plot(scvv[i,vsel][0], scbc[i,vsel][0], drawstyle='steps-post', color='green', alpha=0.5, label='bline c')
            plt.plot(scvv[i,vsel][0], scbd[i,vsel][0], drawstyle='steps-post', color='olive', alpha=0.7, label='bline d', linewidth=2.0)
        elif pmode=='base_a':   #  least aggressive baseline; might slightly compromise lines!
            add += '_bla'
            plt.plot(scvv[i,vsel][0], scsp[i,vsel][0]-scba[i,vsel][0], drawstyle='steps-post', color='blue', alpha=0.99, label='sp-ba')
        elif pmode=='base_b':   #  medium aggressive baseline; might compromise lines!
            plt.plot(scvv[i,vsel][0], scsp[i,vsel][0]-scbb[i,vsel][0], drawstyle='steps-post', color='blue', alpha=0.99, label='sp-bb')
        elif pmode=='base_c':   #  most aggressive baseline; might take away lines!
            plt.plot(scvv[i,vsel][0], scsp[i,vsel][0]-scbc[i,vsel][0], drawstyle='steps-post', color='blue', alpha=0.99, label='sp-bc')
        elif pmode=='base_d':   #  least aggressive baseline with line exclusion
            plt.plot(scvv[i,vsel][0], scsp[i,vsel][0]-scbd[i,vsel][0], drawstyle='steps-post', color='blue', alpha=0.4, label='sp-bd')
            scvvr, scspr = resampleSpectrum((scvv[i,vsel][0]), (scsp[i,vsel][0]-scbd[i,vsel][0]), 4)
            plt.plot(scvvr, scspr, drawstyle='steps-post', color='blue', alpha=0.99, label='rsmp. sp')
        elif pmode=='appl_d':   # default is the least aggressive baseline with line exclusion
            plt.plot(scvv[i,vsel][0], scsp[i,vsel][0]-scbd[i,vsel][0], drawstyle='steps-post', color='blue', alpha=0.1, label='f.-r. spec')
            scvvr, scspr = resampleSpectrum((scvv[i,vsel][0]), (scsp[i,vsel][0]-scbd[i,vsel][0]), 4)
            plt.plot(scvvr, scspr, drawstyle='steps-post', color='blue', alpha=0.99, label='rsmp. sp')
        else:   # default is the least aggressive baseline with line exclusion
            plt.plot(scvv[i,vsel][0], scsp[i,vsel][0]-scbd[i,vsel][0], drawstyle='steps-post', color='blue', alpha=0.1, label='f.-r. spec')
            scvvr, scspr = resampleSpectrum((scvv[i,vsel][0]), (scsp[i,vsel][0]-scbd[i,vsel][0]), 4)
            plt.plot(scvvr, scspr, drawstyle='steps-post', color='blue', alpha=0.99, label='rsmp. sp')
        
        # plot the YS spectrum; lin=0: NII_1, lin=1: NII_2, lin=2: CII_2
#         if plotys:
#             try:
#                 print('plotys')
#                 tsel = np.squeeze(np.argwhere((iscan==yscans) & (scid[i]==yobsids)))
#                 print('1', tsel.shape,lin,yvv.shape, tsel)
#                 if tsel.size>0: 
#                     yvvr, yspr = resampleSpectrum(yvv[tsel,lin,:], ysp[tsel,lin,:], 4)
#                     print('2', yvvr.shape, yspr.shape)
#                     plt.plot(yvvr, yspr, drawstyle='steps-post', color='red', label='YS', alpha=0.6)
#                     print('3')
#                 if debug: print('YS spectrum: ', yscans[tsel], yobsids[tsel], yspr.min(), yspr.mean(), yspr.max())
#             except OSError as err:
#                 print("plotys error: {0}".format(err))
#                 pass
        
        
        
        # legend:
        if (iscan==scans[0]) & (i==0): plt.legend()
                        
        plt.text(ptx0,pty0,'%i: %.3f,%.3f  %i'%(i, scgl[i], scgb[i], scid[i]),size=8)
        plt.ylim(pyrange)
        plt.xlim(pxrange)
        plt.grid()
        plt.gca().tick_params('both', labelsize=8, length=6, width=1.5, tickdir='in')
        if i<n_rows: plt.ylabel(r'$T_A^* (K)$', size=tsz)
        if tidx[i]>n_cols*(n_rows-1)-1: plt.xlabel(r'$v_{LSR} (km/s)$', size=tsz)
        if i==0:
            #bcorr0 = bcorr[vsel]
            scsp0 = scsp[i,vsel][0]
            scrf0 = scr1[i,vsel][0]
            scvv0 = scvv[i,vsel][0]
            sct10 = sct1[i,vsel][0]
            sct20 = sct2[i,vsel][0]
            bcor10 = bcor1[i,vsel][0]
            bcor20 = bcor2[i,vsel][0]
        scspf = scsp[i,vsel][0]
        scrff = scr1[i,vsel][0]
        scvvf = scvv[i,vsel][0]

    if n_sc<n_cols*n_rows: 
        plt.xlabel(r'$v_{LSR} (km/s)$', size=tsz)
        

        
    pdf.savefig(fig) # save figure

    
pdf.close()    
print('Created file: %s'%(outfile))